# Generating Shakespeare

## Setup

We're going to download the collected plays of Shakespeare to use as our data.

Source: http://www.gutenberg.org/cache/epub/100/pg100.txt

The original source was preprocessed to remove sonnets and non-Shakesperean text added by Project Gutenberg.

In [1]:
import os

BASE_DIR = os.getcwd()
DATA_DIR = BASE_DIR + '/data/shakespeare/'

In [41]:
model_path = DATA_DIR + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

In [2]:
data = DATA_DIR + 'gutenberg_shakespeare_modified.txt' # preprocessed

with open(data, 'r') as f:
    text = f.read()
print('corpus length:', len(text))

('corpus length:', 5291227)


In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

('total chars:', 88)


Sometimes it's useful to have a zero value in the dataset, e.g. for padding

In [4]:
chars.insert(0, "\0")

In [5]:
''.join(chars)

'\x00\n\r !"&\'(),-.0123456789:;<?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_`abcdefghijklmnopqrstuvwxyz|}\xbb\xbf\xef'

Map chars to indices and vice versa

In [6]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [7]:
print(char_indices)

{'\x00': 0, ' ': 3, '(': 8, ',': 10, '0': 13, '4': 17, '8': 21, '\xbb': 85, '<': 25, '\xbf': 86, 'D': 30, 'H': 34, 'L': 38, 'P': 42, 'T': 46, 'X': 50, '`': 56, 'd': 60, 'h': 64, 'l': 68, '\xef': 87, 'p': 72, 't': 76, 'x': 80, '|': 83, "'": 7, '3': 16, '7': 20, ';': 24, '?': 26, 'C': 29, 'G': 33, 'K': 37, 'O': 41, 'S': 45, 'W': 49, '[': 53, '_': 55, 'c': 59, 'g': 63, 'k': 67, 'o': 71, 's': 75, 'w': 79, '\n': 1, '"': 5, '&': 6, '.': 12, '2': 15, '6': 19, ':': 23, 'B': 28, 'F': 32, 'J': 36, 'N': 40, 'R': 44, 'V': 48, 'Z': 52, 'b': 58, 'f': 62, 'j': 66, 'n': 70, 'r': 74, 'v': 78, 'z': 82, '\r': 2, '!': 4, ')': 9, '-': 11, '1': 14, '5': 18, '9': 22, 'A': 27, 'E': 31, 'I': 35, 'M': 39, 'Q': 43, 'U': 47, 'Y': 51, ']': 54, 'a': 57, 'e': 61, 'i': 65, 'm': 69, 'q': 73, 'u': 77, 'y': 81, '}': 84}


*idx* converts the Shakepearean text to character indices (based on the *char_indices* mapping above)

In [8]:
idx = [char_indices[c] for c in text]

In [9]:
print(idx[:70])

[87, 85, 86, 45, 29, 31, 40, 31, 23, 2, 1, 44, 71, 77, 75, 65, 68, 68, 71, 70, 24, 3, 42, 57, 74, 65, 75, 24, 3, 32, 68, 71, 74, 61, 70, 59, 61, 24, 3, 39, 57, 74, 75, 61, 65, 68, 68, 61, 75, 2, 1, 2, 1, 2, 1, 27, 29, 46, 3, 35, 12, 3, 45, 29, 31, 40, 31, 3, 14, 12]


In [10]:
''.join(indices_char[i] for i in idx[:70])

'\xef\xbb\xbfSCENE:\r\nRousillon; Paris; Florence; Marseilles\r\n\r\n\r\nACT I. SCENE 1.'

## 3 char model

### Create inputs

Create a list of every 4th character, starting at the 0th, 1st, 2nd, then 3rd characters

In [ ]:
nc=3 # num chars
c1_dat = [idx[i] for i in xrange(0, len(idx)-1-nc, nc)]
c2_dat = [idx[i+1] for i in xrange(0, len(idx)-1-nc, nc)]
c3_dat = [idx[i+2] for i in xrange(0, len(idx)-1-nc, nc)]
c4_dat = [idx[i+3] for i in xrange(0, len(idx)-1-nc, nc)]

In [ ]:
0, len(idx)-1-nc, nc

In [ ]:
len(c1_dat), len(c4_dat)

Out inputs

In [ ]:
import numpy as np

x1 = np.stack(c1_dat)
x2 = np.stack(c2_dat)
x3 = np.stack(c3_dat)

Out output

In [ ]:
y = np.stack(c4_dat)

In [ ]:
x1.shape, y.shape

In [ ]:
n_fac = 42 # number of latent factors (size of embedding matrix)

Create inputs and embedding outputs for each of our 3 character inputs

In [ ]:
from keras.layers import Input, Embedding
from keras.layers.core import Flatten

def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name+'_in')
    emb = Embedding(n_in, n_out, input_length=1, name=name+'_emb')(inp)
    return inp, Flatten()(emb)

In [ ]:
c1_in, c1_emb = embedding_input('c1', vocab_size, n_fac)
c2_in, c2_emb = embedding_input('c2', vocab_size, n_fac)
c3_in, c3_emb = embedding_input('c3', vocab_size, n_fac)

### Create and train model

In [ ]:
n_hidden = 256 # hyperparameter: size of hidden state

![3char](./3char.png)

`dense_in` is the 'green arrow' in the diagram - the layer operation from input to hidden

In [ ]:
from keras.layers.core import Dense

dense_in = Dense(n_hidden, activation='relu')

Our first hidden activation is simply this function applied to the result of the embedding of the first character.

In [ ]:
c1_hidden = dense_in(c1_emb)

`dense_hidden` is the 'orange arrow' from our diagram - the layer operation from hidden to hidden

_Note:_ unsure why the activation for this is `tanh`

In [ ]:
dense_hidden = Dense(n_hidden, activation='tanh')

Our second and third activations sum up the previous hidden state (after applying `dense_hidden`) to the new input state.

In [ ]:
from keras.layers import merge

# merge([new input state, orange arrow from previous hidden state])
c2_hidden = merge([dense_in(c2_emb), dense_hidden(c1_hidden)])
c3_hidden = merge([dense_in(c3_emb), dense_hidden(c2_hidden)])

`dense_out` is the 'blue arrow' from our diagram - the layer operation from hidden to output

In [ ]:
dense_out = Dense(vocab_size, activation='softmax')

The third hidden state is the input to our output layer

In [ ]:
c4_out = dense_out(c3_hidden)

In [ ]:
from keras.models import Model
from keras.optimizers import Adam

model = Model([c1_in, c2_in, c3_in], c4_out)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
model.optimizer.lr=0.000001

In [ ]:
model.summary()

In [ ]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)

In [ ]:
model.optimizer.lr=0.01

In [ ]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)

In [ ]:
model.optimizer.lr=0.000001

In [ ]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)

In [ ]:
model.optimizer.lr=0.01

In [ ]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)

In [ ]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=10)

In [ ]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=10)

In [ ]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=10)

In [ ]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=10)

Let's save the model.

In [ ]:
save1_path = model_path + 'save1.h5'
if not os.path.exists(save1_path):
    model.save_weights(save1_path)
model.load_weights(save1_path)

### Test Model

"`newaxis` is used to increase the dimension of the existing array by one more dimension, when used once" - [source](https://stackoverflow.com/questions/29241056/the-use-of-numpy-newaxis)

In [ ]:
def get_next(m, inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = m.predict(arrs)
    i = np.argmax(p)
    return chars[i]

In [ ]:
get_next(model, 'phi')

In [ ]:
get_next(model, ' th')

In [ ]:
get_next(model, ' an')

## Our first RNN!


### Create inputs

Now let's try predicting char 9 using chars 1-8.

In [37]:
import numpy as np
from keras.layers import Input, Embedding, merge
from keras.layers.core import Flatten, Dense
from keras.models import Model
from keras.optimizers import Adam

In [23]:
nc = 8 # numChars == size of our unrolled RNN
n_fac = 42

For each of 0 through 7, create a list of every 8th character with that starting point. These will be the 8 inputs to our model.

In [12]:
c_in_dat = [[idx[i+n] for i in xrange(0, len(idx)-1-nc, nc)]
           for n in range(nc)]

Then create a list of the next character in each of these series. This will be the labels for our model.

In [13]:
c_out_dat = [idx[i+nc] for i in xrange(0, len(idx)-1-nc, nc)]

In [83]:
xs = [np.stack(c) for c in c_in_dat]

In [17]:
len(xs), xs[0].shape

(8, (661403,))

In [18]:
y = np.stack(c_out_dat)

So each column below is one series of 8 characters from the text:

In [19]:
[xs[n][:nc] for n in range(nc)]

[array([87, 23, 68, 74, 74, 57, 75, 29]),
 array([85,  2, 68, 65, 61, 74,  2, 46]),
 array([86,  1, 71, 75, 70, 75,  1,  3]),
 array([45, 44, 70, 24, 59, 61,  2, 35]),
 array([29, 71, 24,  3, 61, 65,  1, 12]),
 array([31, 77,  3, 32, 24, 68,  2,  3]),
 array([40, 75, 42, 68,  3, 68,  1, 45]),
 array([31, 65, 57, 71, 39, 61, 27, 29])]

...and this is the next character after each sequence:

In [20]:
y[:nc]

array([23, 68, 74, 74, 57, 75, 29, 31])

### Create and train model

In [43]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name+'_in')
    emb = Embedding(n_in, n_out, input_length=1, name=name+'_emb')(inp)
    return inp, Flatten()(emb)

In [44]:
cs = [embedding_input('c'+str(n), vocab_size, n_fac) for n in range(nc)]

In [45]:
n_hidden = 256

"I'd suggest trying the trick I mentioned in the lesson for simple RNNs: using an identity matrix to initialize your hidden state, and use relu instead of tanh." - [Jeremy on forums](http://forums.fast.ai/t/purpose-of-rnns-and-theano/242/5)

In [46]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax')

The embedding of the first character of each sequence goes through `dense_in` to create our first hidden activations.

In [47]:
hidden = dense_in(cs[0][1])

Then for each successive layer, we combine the output of `dense_in` on the next character with the output of `dense_hidden` on the current hidden state to create the new hidden state.

In [48]:
for i in range(1, nc):
    dense = dense_in(cs[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([dense, hidden])

Putting the final hidden state through `dense_out` gives us our output.

In [49]:
out = dense_out(hidden)

Now we can create our model.

In [77]:
model = Model([c[0] for c in cs], out)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
c0 (InputLayer)                  (None, 1)             0                                            
____________________________________________________________________________________________________
c1 (InputLayer)                  (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 1, 42)         3696        c0[0][0]                         
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 1, 42)         3696        c1[0][0]                         
___________________________________________________________________________________________

In [78]:
model.fit(xs, y, batch_size=64, nb_epoch=12)

Epoch 1/12
661403/661403 [==============================] - 89s - loss: 1.6109    
Epoch 2/12
168256/661403 [======>.......................] - ETA: 67s - loss: 1.5958

KeyboardInterrupt: 

### Test Model

In [54]:
def get_next(m, inp):
    arrs = [np.array(char_indices[c])[np.newaxis] for c in inp]
    p = m.predict(arrs)
    return chars[np.argmax(p)]

In [ ]:
get_next(model, 'for thos')

In [ ]:
get_next(model, 'part of ')

In [ ]:
get_next(model, 'queens a')

Here's a helper function for generating `k` additional words (separated by whitespace) in a starter sequence

In [51]:
def get_seq(m, inp, k):
    k_count = 0
    seq = inp
    while k_count < k+1:
        pc = get_next(m, inp)
        seq += pc
        inp = inp[1:] + pc
        if (pc == ' '):
            k_count += 1
    return seq

In [ ]:
get_seq(model, 'queens a', 10)

In [ ]:
get_seq(model, 'part of ', 10)

In [ ]:
get_seq(model, 'for thos', 10)

Model currently seems to 'fixate' on the phrase "the some sore"

In [ ]:
model.fit(xs, y, batch_size=64, nb_epoch=12)

In [55]:
get_seq(model, 'queens a', 10)

'queens and the best with the best with the best with the '

In [56]:
get_seq(model, 'part of ', 10)

'part of the beat the best with the best with the best with '

In [57]:
get_seq(model, 'for thos', 10)

'for thos mey so that she seanon the rest readond and the '

In [52]:
save2_path = model_path + 'save2.h5'
if not os.path.exists(save2_path):
    model.save_weights(save2_path)
model.load_weights(save2_path)

Different 'fixation' on the phrase "the best with"

## Our first RNN with keras!

This is nearly equivalent to the RNN we built ourselves in the previous section.

In [97]:
from keras.models import Sequential
from keras.layers import SimpleRNN

n_hidden, n_fac, nc, vocab_size = (256, 42, 8, 88)

In [98]:
model = Sequential([
        Embedding(vocab_size, n_fac, input_length=nc),
        SimpleRNN(n_hidden, activation='relu', inner_init='identity'),
        Dense(vocab_size, activation='softmax')
    ])
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_11 (Embedding)         (None, 8, 42)         3696        embedding_input_3[0][0]          
____________________________________________________________________________________________________
simplernn_3 (SimpleRNN)          (None, 256)           76544       embedding_11[0][0]               
____________________________________________________________________________________________________
dense_9 (Dense)                  (None, 88)            22616       simplernn_3[0][0]                
Total params: 102856
____________________________________________________________________________________________________


To avoid `IndexError: axis 1 out of bounds [0, 1)`: http://forums.fast.ai/t/lesson-6-discussion/245/70

In [99]:
model.fit(np.concatenate([x[np.newaxis] for x in xs]).T, y, batch_size=64, nb_epoch=12)

Epoch 1/12
661403/661403 [==============================] - 94s - loss: 1.9521    
Epoch 2/12
661403/661403 [==============================] - 95s - loss: 1.5977    
Epoch 3/12
661403/661403 [==============================] - 92s - loss: 1.5132    
Epoch 4/12
661403/661403 [==============================] - 91s - loss: 1.4692    
Epoch 5/12
661403/661403 [==============================] - 87s - loss: 1.4430    
Epoch 6/12
661403/661403 [==============================] - 88s - loss: 1.4235    
Epoch 7/12
661403/661403 [==============================] - 87s - loss: 1.4093    
Epoch 8/12
661403/661403 [==============================] - 91s - loss: 1.3991    
Epoch 9/12
661403/661403 [==============================] - 88s - loss: 1.3907    
Epoch 10/12
661403/661403 [==============================] - 89s - loss: 1.3837    
Epoch 11/12
661403/661403 [==============================] - 88s - loss: 1.3779    
Epoch 12/12
661403/661403 [==============================] - 87s - loss: 1.3727    


In [100]:
def get_next_keras(m, inp):
    idxs = [char_indices[c] for c in inp]
    arrs = np.array(idxs)[np.newaxis,:]
    p = m.predict(arrs)[0]
    return chars[np.argmax(p)]

In [101]:
def get_keras_seq(m, inp, k):
    k_count = 0
    seq = inp
    while k_count < k+1:
        pc = get_next_keras(m, inp)
        seq += pc
        inp = inp[1:] + pc
        if (pc == ' '):
            k_count += 1
    return seq

In [102]:
get_keras_seq(model, 'queens a', 10)

'queens and the sent the sent the sent the sent the sent '

In [103]:
get_keras_seq(model, 'part of ', 10)

'part of the sent the sent the sent the sent the sent the '

In [104]:
get_keras_seq(model, 'for thos', 10)

'for those the sent the sent the sent the sent the sent '

_Fixation_: "the sent"

In [105]:
model.fit(np.concatenate([x[np.newaxis] for x in xs]).T, y, batch_size=64, nb_epoch=12)

Epoch 1/12
661403/661403 [==============================] - 87s - loss: 1.3689    
Epoch 2/12
661403/661403 [==============================] - 90s - loss: 1.3651    
Epoch 3/12
661403/661403 [==============================] - 89s - loss: 1.3620    
Epoch 4/12
661403/661403 [==============================] - 87s - loss: 1.3588    
Epoch 5/12
661403/661403 [==============================] - 89s - loss: 1.3565    
Epoch 6/12
661403/661403 [==============================] - 95s - loss: 1.3541    
Epoch 7/12
661403/661403 [==============================] - 90s - loss: 1.3523    
Epoch 8/12
661403/661403 [==============================] - 87s - loss: 1.3506    
Epoch 9/12
661403/661403 [==============================] - 89s - loss: 1.3479    
Epoch 10/12
661403/661403 [==============================] - 87s - loss: 1.3474    
Epoch 11/12
661403/661403 [==============================] - 88s - loss: 1.3447    
Epoch 12/12
661403/661403 [==============================] - 87s - loss: 1.3440    


In [106]:
get_keras_seq(model, 'queens a', 10)

'queens and the serve me to the serve me to the serve '

In [107]:
get_keras_seq(model, 'part of ', 10)

'part of the serve me to the serve me to the serve me '

In [108]:
get_keras_seq(model, 'for thos', 10)

'for those the serve me to the serve me to the serve '

_Fixation_: "the serve me"

In [ ]:
save3_path = model_path + 'save3.h5'
if not os.path.exists(save3_path):
    model.save_weights(save3_path)
model.load_weights(save3_path)

## Returning sequences

### Create inputs

To use a sequence model, we can leave our input unchanged - but we have to change our output to a sequence.

Here, `c_out_dat` is identical to `c_in_dat`, but moved across 1 character.

In [109]:
c_out_dat = [[idx[i+n] for i in xrange(1, len(idx)-nc, nc)]
            for n in range(nc)]

In [110]:
ys = [np.stack(c) for c in c_out_dat]

Reading down each column shows one set of inputs and outputs

In [111]:
[xs[n][:nc] for n in range(nc)]

[array([87, 23, 68, 74, 74, 57, 75, 29]),
 array([85,  2, 68, 65, 61, 74,  2, 46]),
 array([86,  1, 71, 75, 70, 75,  1,  3]),
 array([45, 44, 70, 24, 59, 61,  2, 35]),
 array([29, 71, 24,  3, 61, 65,  1, 12]),
 array([31, 77,  3, 32, 24, 68,  2,  3]),
 array([40, 75, 42, 68,  3, 68,  1, 45]),
 array([31, 65, 57, 71, 39, 61, 27, 29])]

In [112]:
[ys[n][:nc] for n in range(nc)]

[array([85,  2, 68, 65, 61, 74,  2, 46]),
 array([86,  1, 71, 75, 70, 75,  1,  3]),
 array([45, 44, 70, 24, 59, 61,  2, 35]),
 array([29, 71, 24,  3, 61, 65,  1, 12]),
 array([31, 77,  3, 32, 24, 68,  2,  3]),
 array([40, 75, 42, 68,  3, 68,  1, 45]),
 array([31, 65, 57, 71, 39, 61, 27, 29]),
 array([23, 68, 74, 74, 57, 75, 29, 31])]

### Create and train model

In [113]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax', name='output')

We're going to pass a vectcor of all zeros as our starting point - here's our input layers for that:

In [114]:
inp1 = Input(shape=(n_fac,), name='zeros')
hidden = dense_in(inp1)

In [115]:
outs = []

for i in range(nc):
    dense = dense_in(cs[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([dense, hidden], mode='sum')
    # every layer now has an output
    outs.append(dense_out(hidden))

In [116]:
model = Model([inp1] + [c[0] for c in cs], outs)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
c0 (InputLayer)                  (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 1, 42)         3696        c0[0][0]                         
____________________________________________________________________________________________________
zeros (InputLayer)               (None, 42)            0                                            
____________________________________________________________________________________________________
dense_10 (Dense)                 (None, 256)           11008       zeros[0][0]                      
                                                                   flatten_9[0][0]         

In [117]:
zeros = np.tile(np.zeros(n_fac), (len(xs[0]), 1))
zeros.shape

(661403, 42)

In [118]:
model.fit([zeros]+xs, ys, batch_size=64, nb_epoch=12)

Epoch 1/12
661403/661403 [==============================] - 215s - loss: 16.4319 - output_loss_1: 2.4577 - output_loss_2: 2.2454 - output_loss_3: 2.0558 - output_loss_4: 1.9853 - output_loss_5: 1.9378 - output_loss_6: 1.9226 - output_loss_7: 1.9170 - output_loss_8: 1.9103   
Epoch 2/12
661403/661403 [==============================] - 219s - loss: 15.2879 - output_loss_1: 2.4150 - output_loss_2: 2.1894 - output_loss_3: 1.9444 - output_loss_4: 1.8254 - output_loss_5: 1.7530 - output_loss_6: 1.7278 - output_loss_7: 1.7187 - output_loss_8: 1.7142   
Epoch 3/12
661403/661403 [==============================] - 214s - loss: 15.0585 - output_loss_1: 2.4121 - output_loss_2: 2.1849 - output_loss_3: 1.9291 - output_loss_4: 1.7954 - output_loss_5: 1.7138 - output_loss_6: 1.6823 - output_loss_7: 1.6721 - output_loss_8: 1.6689   
Epoch 4/12
661403/661403 [==============================] - 213s - loss: 14.9454 - output_loss_1: 2.4110 - output_loss_2: 2.1823 - output_loss_3: 1.9221 - output_loss_4: 1.

### Test model

In [119]:
def get_nexts(m, inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict([np.zeros(n_fac)[np.newaxis,:]] + arrs)
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [121]:
get_nexts(model, ' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


[' ', 'h', 'e', 't', ' ', 's', 'n', ' ']

In [ ]:
get_nexts(model, ' part of')

In [122]:
get_nexts(model, 'queens a')

['q', 'u', 'e', 'e', 'n', 's', ' ', 'a']


['u', 'e', 'r', 'n', ' ', ' ', 't', 'n']

### Sequence model with keras

To convert our previous keras model into a sequence model, simply add the `return_sequences=True` parameter, and add `TimeDistributed` around our dense layer.

In [124]:
from keras.layers import TimeDistributed

model = Sequential([
        Embedding(vocab_size, n_fac, input_length=nc),
        SimpleRNN(n_hidden, return_sequences=True, activation='relu', inner_init='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax'))
    ])
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_13 (Embedding)         (None, 8, 42)         3696        embedding_input_4[0][0]          
____________________________________________________________________________________________________
simplernn_5 (SimpleRNN)          (None, 8, 256)        76544       embedding_13[0][0]               
____________________________________________________________________________________________________
timedistributed_1 (TimeDistribute(None, 8, 88)         22616       simplernn_5[0][0]                
Total params: 102856
____________________________________________________________________________________________________


In [125]:
xs[0].shape

(661403,)

In [126]:
x_rnn = np.stack(np.squeeze(xs), axis=1)
y_rnn = np.atleast_3d(np.stack(ys, axis=1))

In [127]:
x_rnn.shape, y_rnn.shape

((661403, 8), (661403, 8, 1))

In [128]:
model.fit(x_rnn, y_rnn, batch_size=64, nb_epoch=8)

Epoch 1/8
661403/661403 [==============================] - 133s - loss: 1.8399   
Epoch 2/8
661403/661403 [==============================] - 134s - loss: 1.6590   
Epoch 3/8
661403/661403 [==============================] - 123s - loss: 1.6273   
Epoch 4/8
661403/661403 [==============================] - 129s - loss: 1.6125   
Epoch 5/8
661403/661403 [==============================] - 123s - loss: 1.6031   
Epoch 6/8
661403/661403 [==============================] - 123s - loss: 1.5967   
Epoch 7/8
661403/661403 [==============================] - 123s - loss: 1.5919   
Epoch 8/8
661403/661403 [==============================] - 123s - loss: 1.5879   


In [129]:
model.fit(x_rnn, y_rnn, batch_size=64, nb_epoch=4)

Epoch 1/4
661403/661403 [==============================] - 126s - loss: 1.5847   
Epoch 2/4
661403/661403 [==============================] - 130s - loss: 1.5823   
Epoch 3/4
661403/661403 [==============================] - 127s - loss: 1.5800   
Epoch 4/4
661403/661403 [==============================] - 124s - loss: 1.5782   


In [132]:
def get_nexts_keras(m, inp):
    idxs = [char_indices[c] for c in inp]
    arrs = np.array(idxs)[np.newaxis,:]
    p = m.predict(arrs)[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [133]:
get_nexts_keras(model, ' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


[' ', 'h', 'e', 's', ' ', 's', 's', ' ']

In [134]:
get_nexts_keras(model, ' part of')

[' ', 'p', 'a', 'r', 't', ' ', 'o', 'f']


[' ', 'r', 'r', 't', ' ', 'o', 'f', ' ']

In [135]:
get_nexts_keras(model, 'queens a')

['q', 'u', 'e', 'e', 'n', 's', ' ', 'a']


['u', 'e', 's', 'n', ' ', ' ', 'a', 'n']

In [ ]:
save4_path = model_path + 'save4.h5'
if not os.path.exists(save4_path):
    model.save_weights(save4_path)
model.load_weights(save4_path)

## Stateful model with keras

## Char RNN

https://github.com/fastai/courses/blob/master/deeplearning1/nbs/char-rnn.ipynb